# Dependencies & Read Data 

In [1]:
#Imports
import pandas as pd
import requests
import gmaps
import re
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
#Read the CSV & Create DF
filtered_df = pd.read_csv("routine_boulder_inspections_filtered.csv")
filtered_df.head()

,Unnamed: 0,facilityId,facilityName,siteAddress,streetNumber,streetDirection,streetName,streetType,streetUnit,city,...,categoryOfFacility,inspectionType,inspectionDate,violationCode,violation,violationPoints,violationType,violationStatus,inspectionScore,location
0,0,FA0003323,RUNZA,1743 MAIN ST,1743,NaN,MAIN,ST,NaN,LONGMONT,...,FULL MENU LIMITED SERVICE,ROUTINE INSPECTION,11/13/2013,01B,Wholesome Free of Spoilage,0,Critical,In,27,"1743 MAIN ST\r\nLONGMONT, CO 80501\r\n(40.1897..."
1,1,FA0003323,RUNZA,1743 MAIN ST,1743,NaN,MAIN,ST,NaN,LONGMONT,...,FULL MENU LIMITED SERVICE,ROUTINE INSPECTION,11/13/2013,15B,Linen Properly Stored,0,Non-Critical,Not Observed,27,"1743 MAIN ST\r\nLONGMONT, CO 80501\r\n(40.1897..."
2,2,FA0003323,RUNZA,1743 MAIN ST,1743,NaN,MAIN,ST,NaN,LONGMONT,...,FULL MENU LIMITED SERVICE,ROUTINE INSPECTION,11/13/2013,06C,Soap and Drying Devices,0,Critical,In,27,"1743 MAIN ST\r\nLONGMONT, CO 80501\r\n(40.1897..."
3,3,FA0003323,RUNZA,1743 MAIN ST,1743,NaN,MAIN,ST,NaN,LONGMONT,...,FULL MENU LIMITED SERVICE,ROUTINE INSPECTION,2/6/2015,03G,Adequate Equipment to Maintain Food Temperatures,0,Critical,In,0,"1743 MAIN ST\r\nLONGMONT, CO 80501\r\n(40.1897..."
4,4,FA0003323,RUNZA,1743 MAIN ST,1743,NaN,MAIN,ST,NaN,LONGMONT,...,FULL MENU LIMITED SERVICE,ROUTINE INSPECTION,11/13/2013,03C,Hot Hold at 135 Degrees Farenheit or Greater,0,Critical,In,27,"1743 MAIN ST\r\nLONGMONT, CO 80501\r\n(40.1897..."


# Filtering

In [3]:
#determine which field or fields should be used in geocoding.
filtered_df.isnull().sum()
# siteAddress,city,state

Unnamed: 0                 0
facilityId                 0
facilityName               0
siteAddress                0
streetNumber             124
streetDirection       170538
streetName                 0
streetType              5003
streetUnit            150059
city                       0
state                      0
zip                        0
typeOfFacility             0
categoryOfFacility         0
inspectionType             0
inspectionDate             0
violationCode              0
violation                  0
violationPoints            0
violationType          16713
violationStatus         1476
inspectionScore            0
location                   0
dtype: int64

In [7]:
#drop 'unnamed' column
unnamed_dropped = filtered_df.drop('Unnamed: 0',1)

# concatenate address, and add country, add to a new column
unnamed_dropped["concat_address"] = unnamed_dropped["siteAddress"] + " " + unnamed_dropped["city"] + " " + unnamed_dropped["state"] + " USA"

C:\Users\natal\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


In [11]:
# New dataframe containing only UNIQUE addresses
unique_rest = unnamed_dropped.drop_duplicates(subset ="concat_address")
# unnamed_dropped
unique_rest

,facilityId,facilityName,siteAddress,streetNumber,streetDirection,streetName,streetType,streetUnit,city,state,...,inspectionType,inspectionDate,violationCode,violation,violationPoints,violationType,violationStatus,inspectionScore,location,concat_address
0,FA0003323,RUNZA,1743 MAIN ST,1743,NaN,MAIN,ST,NaN,LONGMONT,CO,...,ROUTINE INSPECTION,11/13/2013,01B,Wholesome Free of Spoilage,0,Critical,In,27,"1743 MAIN ST\r\nLONGMONT, CO 80501\r\n(40.1897...",1743 MAIN ST LONGMONT CO USA
124,FA0000616,FRANKS CHOP HOUSE,921 WALNUT ST,921,NaN,WALNUT,ST,NaN,BOULDER,CO,...,ROUTINE INSPECTION,4/11/2013,02E,Smoking Eating Drinking,0,Critical,In,20,"921 WALNUT ST\r\nBOULDER, CO 80302\r\n(40.0162...",921 WALNUT ST BOULDER CO USA
126,FA0004494,ROASTED TOAD BBQ THE,229 HWY 119,229,NaN,HWY 119,NaN,NaN,NEDERLAND,CO,...,ROUTINE INSPECTION,5/29/2014,02C,Hands Washed As Needed,0,Critical,In,45,"229 HWY 119\r\nNEDERLAND, CO 80466",229 HWY 119 NEDERLAND CO USA
127,FA0003893,FAIR ISLE COFFEE CO @ LGMT FARM MRKT,237 COLLYER ST,237,NaN,COLLYER,ST,NaN,Longmont,CO,...,ROUTINE INSPECTION,4/26/2014,08B,Properly Labeled,0,Critical,In,5,"237 COLLYER ST\r\nLongmont, CO 80501\r\n(40.16...",237 COLLYER ST Longmont CO USA
128,FA0003472,TODS ESPRESSO CAFE,6558 LOOKOUT RD,6558,NaN,LOOKOUT,RD,NaN,BOULDER,CO,...,ROUTINE INSPECTION,6/20/2019,FC51,Plumbing installed; proper backflow devices,10,NaN,Not Observed,10,"6558 LOOKOUT RD\r\nBOULDER, CO 80301\r\n(40.07...",6558 LOOKOUT RD BOULDER CO USA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154560,FA0005279,VOLCAN AZUL (MOBILE),600 COFFMAN ST,600,NaN,COFFMAN,ST,NaN,Longmont,CO,...,REGULAR INSPECTION (30 DAYS),9/5/2018,14E,Ventilation,0,Non-Critical,Not Observed,51,"600 COFFMAN ST\r\nLongmont, CO 80501\r\n(40.16...",600 COFFMAN ST Longmont CO USA
154854,FA0005473,ALTAVITA INDEPENDENT LIVING,1002 ALTA VITA CT,1002,NaN,ALTA VITA,CT,NaN,LONGMONT,CO,...,REGULAR INSPECTION (30 DAYS),7/3/2019,FC29,Compliance with variance / specialized process...,20,Critical,Not Applicable,60,"1002 ALTA VITA CT\r\nLONGMONT, CO 80503\r\n(40...",1002 ALTA VITA CT LONGMONT CO USA
155339,FA0005424,BIG LOTS #4685,2151 MAIN ST,2151,NaN,MAIN,ST,NaN,Longmont,CO,...,REGULAR INSPECTION (30 DAYS),5/10/2019,FC03,"Management, food employee and conditional empl...",5,Critical,In,10,"2151 MAIN ST\r\nLongmont, CO 80501\r\n(40.1968...",2151 MAIN ST Longmont CO USA
155682,FA0005494,ROSATIS CHICAGO PIZZA,1067 COURTESY RD,1067,NaN,COURTESY,RD,NaN,Louisville,CO,...,REGULAR INSPECTION (30 DAYS),10/1/2019,FC38,"Insects, rodents, & animals not present",10,NaN,In,15,"1067 COURTESY RD\r\nLouisville, CO 80027\r\n(3...",1067 COURTESY RD Louisville CO USA


# Pulled & Cleaned Geolocation Data from original Dataset

In [43]:
#Found the Geolocation Data in the original dataset
value = unique_rest['location'].values[0]
value.split('\r\n')[2]

'(40.189746, -105.102332)'

In [34]:
# Create column for coords pulled from dataset
unique_rest["split_coords"] = ""

C:\Users\natal\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [41]:
# Split the location field and dropped the document coordinates into their own column
for index, row in unique_rest.iterrows():
    
    #split the data in the location column
    messy = unique_rest['location'].values(index)
    coords = messy.split('\r\n')[2]
    
#        messy_coords = unique_rest['location'].values
#     coords = values.split('\r\n')[2]
    
    #remove the characters
    clean_coords = coords.replace("(","").replace(")","")
    
    #drop the coor
    unique_rest.loc[index, "split_coords"] = clean_coords

unique_rest.head()

TypeError: 'numpy.ndarray' object is not callable

# Geolocated Addresses w/ Google API

In [26]:
# struture #API Call & test on a single address
params = {
    "address": "1743 MAIN ST LONGMONT CO USA",
    "key": g_key
}

base_url = "https://maps.googleapis.com/maps/api/geocode/json"

latlong = requests.get(base_url, params=params).json()

latlong ["results"][0]["geometry"]["location"]['lat']
latlong ["results"][0]["geometry"]["location"]['lng']

-105.1026674

In [28]:
#create column to hold Google Geoloation Data
unique_rest["google_coords"] = ""
# unique_rest.head()

C:\Users\natal\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [29]:
#create forloop(s) to extract each piece of data and drop into the url line by line.
#set params dictionary
params = {
    "key": g_key
}

#iterate through that dataframe!
for index, row in unique_rest.iterrows():
    # Get the Address from the dataframe
    address = row["concat_address"]
    params["address"] = address

    # Set the base URL to grab JSON data
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    # Make request and retrieve the JSON data from the search. 
    latlong = requests.get(base_url, params=params).json()
    
    #Set Lat/Long variables
    lat = latlong["results"][0]["geometry"]["location"]['lat']
    long = latlong ["results"][0]["geometry"]["location"]['lng']
    combined = f"{lat},{long}"
    # Get coordinates & store, if not found, skip! 
    try: 
        unique_rest.loc[index, "google_coords"] = combined
    except (IndexError):
        print(f"{address} not found, skip") 


C:\Users\natal\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,facilityId,facilityName,siteAddress,streetNumber,streetDirection,streetName,streetType,streetUnit,city,state,...,violationCode,violation,violationPoints,violationType,violationStatus,inspectionScore,location,concat_address,split_coords,google_coords
0,FA0003323,RUNZA,1743 MAIN ST,1743,NaN,MAIN,ST,NaN,LONGMONT,CO,...,01B,Wholesome Free of Spoilage,0,Critical,In,27,"1743 MAIN ST\r\nLONGMONT, CO 80501\r\n(40.1897...",1743 MAIN ST LONGMONT CO USA,"40.189746, -105.102332","40.1897717,-105.1026674"
124,FA0000616,FRANKS CHOP HOUSE,921 WALNUT ST,921,NaN,WALNUT,ST,NaN,BOULDER,CO,...,02E,Smoking Eating Drinking,0,Critical,In,20,"921 WALNUT ST\r\nBOULDER, CO 80302\r\n(40.0162...",921 WALNUT ST BOULDER CO USA,"40.189746, -105.102332","40.0165533,-105.2828453"
126,FA0004494,ROASTED TOAD BBQ THE,229 HWY 119,229,NaN,HWY 119,NaN,NaN,NEDERLAND,CO,...,02C,Hands Washed As Needed,0,Critical,In,45,"229 HWY 119\r\nNEDERLAND, CO 80466",229 HWY 119 NEDERLAND CO USA,"40.189746, -105.102332","39.9631805,-105.5095941"
127,FA0003893,FAIR ISLE COFFEE CO @ LGMT FARM MRKT,237 COLLYER ST,237,NaN,COLLYER,ST,NaN,Longmont,CO,...,08B,Properly Labeled,0,Critical,In,5,"237 COLLYER ST\r\nLongmont, CO 80501\r\n(40.16...",237 COLLYER ST Longmont CO USA,"40.189746, -105.102332","40.1630767,-105.0985565"
128,FA0003472,TODS ESPRESSO CAFE,6558 LOOKOUT RD,6558,NaN,LOOKOUT,RD,NaN,BOULDER,CO,...,FC51,Plumbing installed; proper backflow devices,10,NaN,Not Observed,10,"6558 LOOKOUT RD\r\nBOULDER, CO 80301\r\n(40.07...",6558 LOOKOUT RD BOULDER CO USA,"40.189746, -105.102332","40.0721707,-105.2008731"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154560,FA0005279,VOLCAN AZUL (MOBILE),600 COFFMAN ST,600,NaN,COFFMAN,ST,NaN,Longmont,CO,...,14E,Ventilation,0,Non-Critical,Not Observed,51,"600 COFFMAN ST\r\nLongmont, CO 80501\r\n(40.16...",600 COFFMAN ST Longmont CO USA,"40.189746, -105.102332","40.1692156,-105.1034823"
154854,FA0005473,ALTAVITA INDEPENDENT LIVING,1002 ALTA VITA CT,1002,NaN,ALTA VITA,CT,NaN,LONGMONT,CO,...,FC29,Compliance with variance / specialized process...,20,Critical,Not Applicable,60,"1002 ALTA VITA CT\r\nLONGMONT, CO 80503\r\n(40...",1002 ALTA VITA CT LONGMONT CO USA,"40.189746, -105.102332","40.1494709,-105.1364528"
155339,FA0005424,BIG LOTS #4685,2151 MAIN ST,2151,NaN,MAIN,ST,NaN,Longmont,CO,...,FC03,"Management, food employee and conditional empl...",5,Critical,In,10,"2151 MAIN ST\r\nLongmont, CO 80501\r\n(40.1968...",2151 MAIN ST Longmont CO USA,"40.189746, -105.102332","40.1972884,-105.1026221"
155682,FA0005494,ROSATIS CHICAGO PIZZA,1067 COURTESY RD,1067,NaN,COURTESY,RD,NaN,Louisville,CO,...,FC38,"Insects, rodents, & animals not present",10,NaN,In,15,"1067 COURTESY RD\r\nLouisville, CO 80027\r\n(3...",1067 COURTESY RD Louisville CO USA,"40.189746, -105.102332","39.9803249,-105.1284121"


# Export Unique Dataset with Geocoding Data

In [32]:
# Export unique address dataset
output_data_unique = "unique_restaurants_geocoded.csv"
unique_rest.to_csv(output_data_unique, index="False")

# Return?

In [ ]:
#merge with the original dataset?

In [ ]:
#export combined CSV